**Revision previa del modelo de redes neuronales**

In [1]:
import pandas
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from azureml.core import Workspace, Datastore, Dataset

In [2]:
subscription_id = '<subscription_id>'
resource_group = 'myMLresources'
workspace_name = 'myMLworkspace'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='iris.csv')
dataframe = dataset.to_pandas_dataframe()
print(dataframe.columns)
print(dataframe.head())
print(dataframe.tail())
print(dataframe.shape)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E6BRLUC85 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width',
       'variety'],
      dtype='object')
   sepal.length  sepal.width  petal.length  petal.width variety
0           5.1          3.5           1.4          0.2  Setosa
1           4.9          3.0           1.4          0.2  Setosa
2           4.7          3.2           1.3          0.2  Setosa
3           4.6          3.1           1.5          0.2  Setosa
4           5.0          3.6           1.4          0.2  Setosa
     sepal.length  sepal.width  petal.length  petal.width    variety
145           6.7          3.0           5.2          2.3  Virginica
146           6.3

In [3]:
print(dataframe['variety'].unique())
dataframe['variety'] = dataframe['variety'].replace(['Setosa'], 0)
dataframe['variety'] = dataframe['variety'].replace(['Versicolor'], 1)
dataframe['variety'] = dataframe['variety'].replace(['Virginica'], 2)
print(dataframe['variety'].unique())

data = dataframe.values

['Setosa' 'Versicolor' 'Virginica']
[0 1 2]


In [12]:
ws = Workspace.from_config()
datastore = Datastore.get(ws, 'workspaceblobstore')
dataset = Dataset.Tabular.register_pandas_dataframe(dataframe, datastore, "dataset_procesed.csv", show_progress=True)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/0c78245e-8cd0-4d8a-88ee-b149ad3072d4/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [5]:
x_train, x_test, y_train, y_test = train_test_split(data[:,0:4], data[:,4], test_size=0.33, random_state=42)

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(4,)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 512)               2560      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
Total params: 4,099
Trainable params: 4,099
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=600, verbose=0)

In [10]:
model.evaluate(x_test, y_test)

50/50 [==============================] - 0s 2ms/sample - loss: 0.0419 - accuracy: 0.9800


[0.04190216675400734, 0.98]

In [11]:
curr_Output=model.predict(x_test)
model.evaluate(x_test, y_test)

curr_Output=model.predict(x_test)

Out_test_CM=np.zeros([50,1],dtype=np.float64)
for ii in range(0,50):
    Out_test_CM[ii] = np.argmax(curr_Output[ii,:])

# Calculo de la matriz de confusion
cnf_matrix = confusion_matrix(y_test, Out_test_CM)
np.set_printoptions(precision=2)

cm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
print(cm)

50/50 [==============================] - 0s 84us/sample - loss: 0.0419 - accuracy: 0.9800
[[1.   0.   0.  ]
 [0.   0.93 0.07]
 [0.   0.   1.  ]]
